In [1]:
from pathlib import Path
from os.path import expanduser
from os import path
import numpy as np
import sys

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models
from torchsummary import summary
from torchvision import transforms, datasets
from torch.optim import lr_scheduler
import os



In [2]:
root_folder = os.path.dirname(os.getcwd())
path_to_train_data = path.join(root_folder,'data/results/train_data_re_id.npy');
path_to_labels = path.join(root_folder,'data/results/train_labels_re_id.npy');
#path_to_mean_std = path.join(root_folder,'data/train_data/synthesized/mean_std.npy');

sys.path.append(root_folder)
from custom_resnet import CustomResnet as cnn

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size = 15000;
nb_of_epochs = 550;
logging_interval = 30
dataset_divider = 0.2

In [4]:
data = np.load(path_to_train_data);
labels = np.load(path_to_labels);
torch_data = torch.tensor(data).float()
torch_labels = torch.tensor(labels).int()
nb_of_classes = torch_labels.unique().size()[0]

In [5]:
print(nb_of_classes)

499


In [8]:
net = cnn.ft_net([2, 2, 2, 2], nb_of_classes, 0);
net.to(device)

TypeError: resnet18() takes 0 positional arguments but 2 were given

In [ ]:
# vertical flip
augmented_data = torch.cat((torch_data, torch_data * -1), 0)
augmented_labels = torch.cat((torch_labels, torch_labels), 1)




In [ ]:
if (len(augmented_labels.size()) == 2):
    augmented_labels = torch.squeeze(augmented_labels)

In [ ]:
augmented_labels.size()

In [ ]:
train_data, test_data = cnn.GenerateTrainAndTestDataset(augmented_data, augmented_labels, 0.2)


In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=8)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=8)





In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)
for i in range(nb_of_epochs):  # loop over the dataset multiple times
  loss = cnn.Train(net, device, train_loader, optimizer, criterion, exp_lr_scheduler, i, logging_interval)
  model_name = 'models/re_id/resnet18_num_classes_' + str(nb_of_classes) + "_epoch_" + str(i) + '.pt';
  path_to_model = path.join(root_folder, model_name)
  torch.save({'epoch': i,'model_state_dict': net.state_dict(), 'optimizer_state_dict': optimizer.state_dict(), 'loss': loss,}, path_to_model)
  cnn.Test(net, device, criterion, train_loader)
  cnn.Test(net, device, criterion, test_loader)

In [ ]:
cnn.Test(net, device, criterion, test_loader)